In [1]:
import os
import sys
# sys.path.append("../virtuosoNet")
# sys.path.append("C:\\Users\\fosca\\Desktop\\CNAM\\midi_beat_tracking\\FMP_0.1.0")

from pathlib import Path
import pandas as pd
import music21 as m21
import pretty_midi as pm
from  matplotlib import pyplot as plt
# import LibFMP.B
# import LibFMP.C2
# import LibFMP.C6
%matplotlib inline
import numpy as np
import math
import pandas as pd
import librosa
import soundfile

import IPython.display as ipd

In [31]:
def cleaned_m2m_df(midi2midi_path):
    m2mdf = pd.read_csv(midi2midi_path,sep="\t",skiprows=1,header=None, index_col=False)
    m2mdf.columns =["alignID", "alignOntime", "alignSitch" ,"alignPitch", "alignOnvel", "refID", "refOntime", "refSitch", "refPitch", "refOnvel","empty"]
    #drop the last column "empty"
    m2mdf = m2mdf.drop(columns=["empty"])
    #drop the rows where the alignOntime is -1 (note not existing in both files midi)
    return m2mdf[m2mdf["alignOntime"] != -1]

def generate_performance_annotations(m2m_df, cl_quant_ann):
    #create the list with the time of all annotations
    quant_ann_time_list = cl_quant_ann["time"].tolist()
    quant_ann_type_list = cl_quant_ann["type"].tolist()
    perf_ann_list = []
    for i,at in enumerate(quant_ann_time_list):
        p_ons=list(m2m_df[(m2m_df["refOntime"]>at-0.020) & (m2m_df["refOntime"]<at+0.020)]["alignOntime"]) #40 ms accepted window
        if len(p_ons) == 0: #no note matching the ann position in the performance
            perf_ann_list.append(None)
            print("missing corrispondency at quant time", at)
        else:
            perf_ann_list.append(np.mean(p_ons))

    # clean the values that are smaller than the value before
    for ind,at in enumerate(perf_ann_list[1:]):
        if (ind >0) and (at is not None) and (perf_ann_list[ind-1] is not None) and (at < perf_ann_list[ind-1]) :
            print("Unexpected value",db,"smaller than the previous",perf_ann_list[ind-1])
            perf_ann_list[ind] = None
    
    #save the index for the none values
    none_indices = [index for index,value in enumerate(perf_ann_list) if value is None]
            
    #interpolate to fill the None values
    filled_at = pd.Series(perf_ann_list).interpolate().tolist()
    
    #associate the type of annotation
    assert(len(quant_ann_type_list) == len(filled_at))
    perd_ann_dict = dict(zip(filled_at, quant_ann_type_list))
    #add the Warning for the None values
    for val in none_indices:
        perd_ann_dict[filled_at[val]] += "W"
    
    return perd_ann_dict
    
def beat_downbeat_to_text(annotations,out_path):
    path = Path(out_path)
    #produce the txt
    file_content = ""
    for a in annotations:
        file_content+= str(a) + "\t" + str(a) + "\t" + annotations[a] + "\n"
    with open(out_path,"w") as f:
        f.write(file_content)
        
def annotations_to_audio(midi_path, annotations, out_path):
    midi_data = pm.PrettyMIDI(str(midi_path))
    audio_fs = 11500
    beat = [a for a in annotations if annotations[a][0] == "b"]
    downbeat = [a for a in annotations if annotations[a][0] == "d"]
    midi_audio = midi_data.synthesize(fs=audio_fs)
    audio_beat = librosa.clicks(beat, sr=audio_fs, click_freq=900, length=len(midi_audio))
    audio_downbeat = librosa.clicks(downbeat, sr=audio_fs, click_freq=1200, length=len(midi_audio))
    soundfile.write(out_path,audio_beat + audio_downbeat+ midi_audio,audio_fs)
    
def perform_row_to_annotations(row):
    print("Processing opus", row.folder)
    #load the quantized annotations
    ann_quant_df = pd.read_csv(Path(row.folder,"ann_quant_cleaned.txt"),header=None, names=["time","time2","type"],sep='\t')
    #load the m2m alignment
    m2m = cleaned_m2m_df(row["midi2midi_alignment_path"])
    #generate the annotations
    ann_dict = generate_performance_annotations(m2m, ann_quant_df)
    #write the annotations to txt
    beat_downbeat_to_text(ann_dict,Path(row.folder,row.performer + "_ann.txt"))
    #write the audio
    annotations_to_audio(str(Path(row.performed_midi_path)),ann_dict,Path(row.folder,row.performer + "_click.wav"))
    print("done")

In [32]:
df = pd.read_pickle("performance_dataframe.pkl")
df

,author,folder,midi2midi_alignment_path,performed_midi_path,performer,score2midi_alignment,score_midi_path,score_xml_path,title
0,Bach,Bach/Fugue/bwv_846,Bach/Fugue/bwv_846/Shi05_infer_corresp.txt,Bach/Fugue/bwv_846/Shi05.mid,Shi05,Bach/Fugue/bwv_846/Shi05_infer_match.txt,Bach/Fugue/bwv_846/midi_cleaned.mid,Bach/Fugue/bwv_846/musicxml_cleaned.musicxml,Fugue_bwv_846
1,Bach,Bach/Fugue/bwv_848,Bach/Fugue/bwv_848/Denisova06_infer_corresp.txt,Bach/Fugue/bwv_848/Denisova06.mid,Denisova06,Bach/Fugue/bwv_848/Denisova06_infer_match.txt,Bach/Fugue/bwv_848/midi_cleaned.mid,Bach/Fugue/bwv_848/musicxml_cleaned.musicxml,Fugue_bwv_848
2,Bach,Bach/Fugue/bwv_848,Bach/Fugue/bwv_848/Lee01_infer_corresp.txt,Bach/Fugue/bwv_848/Lee01.mid,Lee01,Bach/Fugue/bwv_848/Lee01_infer_match.txt,Bach/Fugue/bwv_848/midi_cleaned.mid,Bach/Fugue/bwv_848/musicxml_cleaned.musicxml,Fugue_bwv_848
3,Bach,Bach/Fugue/bwv_848,Bach/Fugue/bwv_848/LeeSH01_infer_corresp.txt,Bach/Fugue/bwv_848/LeeSH01.mid,LeeSH01,Bach/Fugue/bwv_848/LeeSH01_infer_match.txt,Bach/Fugue/bwv_848/midi_cleaned.mid,Bach/Fugue/bwv_848/musicxml_cleaned.musicxml,Fugue_bwv_848
4,Bach,Bach/Fugue/bwv_848,Bach/Fugue/bwv_848/Lin04_infer_corresp.txt,Bach/Fugue/bwv_848/Lin04.mid,Lin04,Bach/Fugue/bwv_848/Lin04_infer_match.txt,Bach/Fugue/bwv_848/midi_cleaned.mid,Bach/Fugue/bwv_848/musicxml_cleaned.musicxml,Fugue_bwv_848
5,Bach,Bach/Fugue/bwv_848,Bach/Fugue/bwv_848/Lou01_infer_corresp.txt,Bach/Fugue/bwv_848/Lou01.mid,Lou01,Bach/Fugue/bwv_848/Lou01_infer_match.txt,Bach/Fugue/bwv_848/midi_cleaned.mid,Bach/Fugue/bwv_848/musicxml_cleaned.musicxml,Fugue_bwv_848
6,Bach,Bach/Fugue/bwv_848,Bach/Fugue/bwv_848/MiyashitaM01_infer_corresp.txt,Bach/Fugue/bwv_848/MiyashitaM01.mid,MiyashitaM01,Bach/Fugue/bwv_848/MiyashitaM01_infer_match.txt,Bach/Fugue/bwv_848/midi_cleaned.mid,Bach/Fugue/bwv_848/musicxml_cleaned.musicxml,Fugue_bwv_848
7,Bach,Bach/Fugue/bwv_848,Bach/Fugue/bwv_848/Mizumoto03_infer_corresp.txt,Bach/Fugue/bwv_848/Mizumoto03.mid,Mizumoto03,Bach/Fugue/bwv_848/Mizumoto03_infer_match.txt,Bach/Fugue/bwv_848/midi_cleaned.mid,Bach/Fugue/bwv_848/musicxml_cleaned.musicxml,Fugue_bwv_848
8,Bach,Bach/Fugue/bwv_848,Bach/Fugue/bwv_848/SunY01_infer_corresp.txt,Bach/Fugue/bwv_848/SunY01.mid,SunY01,Bach/Fugue/bwv_848/SunY01_infer_match.txt,Bach/Fugue/bwv_848/midi_cleaned.mid,Bach/Fugue/bwv_848/musicxml_cleaned.musicxml,Fugue_bwv_848
9,Bach,Bach/Fugue/bwv_848,Bach/Fugue/bwv_848/Zhou01_infer_corresp.txt,Bach/Fugue/bwv_848/Zhou01.mid,Zhou01,Bach/Fugue/bwv_848/Zhou01_infer_match.txt,Bach/Fugue/bwv_848/midi_cleaned.mid,Bach/Fugue/bwv_848/musicxml_cleaned.musicxml,Fugue_bwv_848


In [33]:
song = df.iloc[19]
perform_row_to_annotations(song)
song

Processing opus Bach/Fugue/bwv_857
missing corrispondency at quant time 5.5
missing corrispondency at quant time 12.0
missing corrispondency at quant time 113.5
done


author                                                              Bach
folder                                                Bach/Fugue/bwv_857
midi2midi_alignment_path      Bach/Fugue/bwv_857/ToA01_infer_corresp.txt
performed_midi_path                         Bach/Fugue/bwv_857/ToA01.mid
performer                                                          ToA01
score2midi_alignment            Bach/Fugue/bwv_857/ToA01_infer_match.txt
score_midi_path                      Bach/Fugue/bwv_857/midi_cleaned.mid
score_xml_path              Bach/Fugue/bwv_857/musicxml_cleaned.musicxml
title                                                      Fugue_bwv_857
Name: 19, dtype: object

In [26]:
#load the m2m and check if it's correct